In [3]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
from sklearn.model_selection import train_test_split
from pyspark.sql import SparkSession, functions
from pyspark.ml.classification import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator
import mlflow

spark = SparkSession.builder.getOrCreate()

lakehouse_path = "abfss://d9f37163-8b23-41c7-9b52-2a5439f8a766@onelake.dfs.fabric.microsoft.com/904cdc98-c4a5-48fd-8773-bffb959ffd1f/Tables/"

df_sprints = spark.read.format("delta").load(lakehouse_path + "sprints")
df_features = spark.read.format("delta").load(lakehouse_path + "features")
df_developers = spark.read.format("delta").load(lakehouse_path + "developers")

df_sprints_completed = df_sprints.filter(df_sprints.SprintStatus.isin(['Completed', 'Failed']))

df = df_sprints_completed.join(df_features, "SprintID").alias('a').join(df_developers.alias('b'), functions.col("b.EmployeeID") == functions.col('a.AssigneeID'))
grouped_df = df.groupBy("sprintID").agg(
    functions.count("AssigneeID").alias("developers"),
    functions.count("FeaturePoints").alias("features"),
    functions.last("SprintStatus").alias("status")
)
assembler = VectorAssembler(inputCols=["developers", "features"], outputCol="efforts")
grouped_df = assembler.transform(grouped_df)
grouped_df = grouped_df.withColumn("status", functions.when(grouped_df["status"] == "Completed", 1).otherwise(0))
lr = LogisticRegression(featuresCol="efforts", labelCol="status")
model = lr.fit(grouped_df)

with mlflow.start_run(run_id="a3630e91-5681-41ca-9a3b-7c30ae3d8c5c") as run:
    mlflow.spark.log_model(model, "project_dashboard")


StatementMeta(, 9f2ccb81-e394-46c5-9fb6-d5c89c85b47f, 5, Finished, Available)

2024-03-04:07:19:43,56 WARNING  [tracking_store.py:153] log_inputs not supported


2024/03/04 07:20:00 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /tmp/tmpipd9uzbh/model, flavor: spark), fall back to return ['pyspark==3.4.1']. Set logging level to DEBUG to see the full traceback.
